# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

TEXT USED TO IMPORT DATA

$ mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray 

In [5]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [6]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [7]:
# confirm that our new database was created
db_list = mongo.list_database_names()
if "uk_food" in db_list:
    print("The database exists.")
else:
    print("The database does not exist.")


The database exists.


In [8]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [9]:
# review the collections in our new database
collections = db.list_collection_names()
pprint(collections)


['establishments']


In [10]:
# review a document in the establishments collection
establishment = db.establishments.find_one()
pprint(establishment)


{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'Latitude': None,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'Longitude': None,
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65d19833ea4c4fca27dd62bf'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel

In [11]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [12]:
# New restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "", 
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "1446A Plaistow Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebsite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.03834000",
        "latitude": "51.4914200"
    },
    "RightToReply": "",
    "Distance": "4623.973228747176",
    "NewRatingPending": True
}

In [13]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",  
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "1446A Plaistow Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebsite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.03834000",
        "latitude": "51.4914200"
    },
    "RightToReply": "",
    "Distance": "4623.973228747176",
    "NewRatingPending": True
}

In [15]:
# Insert the new restaurant into the collection
db = mongo['uk_food']
establishments = db['establishments']
insert_result = establishments.insert_one(new_restaurant)


In [16]:
# Check that the new restaurant was inserted
if insert_result.acknowledged:
    print("New restaurant inserted with ID:", insert_result.inserted_id)


New restaurant inserted with ID: 65da5df4434c77ee5efc4483


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [17]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = establishments.find_one(
    {'BusinessType': 'Restaurant/Cafe/Canteen'},
    {'BusinessTypeID': 1, 'BusinessType': 1, '_id': 0}
)


3. Update the new restaurant with the `BusinessTypeID` you found.

In [29]:
# Update the new restaurant with the correct BusinessTypeID
update_result = establishments.update_one(
    {'BusinessName': 'Penang Flavours'},
    {'$set': {'BusinessTypeID': business_type['BusinessTypeID']}}
)


In [30]:
# Confirm that the new restaurant was updated
if update_result.modified_count > 0:
    print("New restaurant was updated.")
else:
    print("No changes were made to the restaurant.")

New restaurant was updated.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [31]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({'LocalAuthorityName': 'Dover'})


In [32]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({'LocalAuthorityName': 'Dover'})


In [33]:
# Check if any remaining documents include Dover
dover_exists = establishments.find_one({'LocalAuthorityName': 'Dover'})
if dover_exists:
    print("There are still documents with Dover.")
else:
    print("No documents with Dover found.")


No documents with Dover found.


In [34]:
# Check that other documents remain with 'find_one'
# Check that other documents remain with 'find_one'
document = establishments.find_one()
if document:
    print("A document was found:", document)
else:
    print("No documents found.")


A document was found: {'_id': ObjectId('65d19833ea4c4fca27dd62bf'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': 5, 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:0

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [35]:
# Change the data type from String to Decimal for longitude and latitude
update_lat_lng_result = establishments.update_many(
    {},
    [{'$set': {
        'Latitude': {'$toDecimal': '$Latitude'},
        'Longitude': {'$toDecimal': '$Longitude'}
    }}]
)


Use `update_many` to convert `RatingValue` to integer numbers.

In [25]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [26]:
# Change the data type from String to Integer for RatingValue
update_rating_result = establishments.update_many(
    {},
    [{'$set': {'RatingValue': {'$toInt': '$RatingValue'}}}]
)


In [27]:
# Check that the coordinates and rating value are now numbers
sample_document = establishments.find_one({}, {'Latitude': 1, 'Longitude': 1, 'RatingValue': 1})
pprint(sample_document)


{'Latitude': None,
 'Longitude': None,
 'RatingValue': 5,
 '_id': ObjectId('65d19833ea4c4fca27dd62bf')}
